In [1]:
from mlflow import MlflowClient
from pprint import pprint

client = MlflowClient(tracking_uri="http://127.0.0.1:8080")

# Search experiments without providing query terms behaves effectively as a 'list' action
all_experiments = client.search_experiments()

print(all_experiments)

# Extract the experiment name and lifecycle_stage
default_experiment = [
    {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage}
    for experiment in all_experiments
    if experiment.name == "Default"
][0]

pprint(default_experiment)

[<Experiment: artifact_location='mlflow-artifacts:/123960551536115947', creation_time=1700088037390, experiment_id='123960551536115947', last_update_time=1700088037390, lifecycle_stage='active', name='Apple_Models', tags={'mlflow.note.content': 'This is the grocery forecasting project. This '
                        'experiment contains the produce models for apples.',
 'project_name': 'grocery-forecasting',
 'project_quarter': 'Q3-2023',
 'store_dept': 'produce',
 'team': 'stores-ml'}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1700088001087, experiment_id='0', last_update_time=1700088001087, lifecycle_stage='active', name='Default', tags={}>]
{'lifecycle_stage': 'active', 'name': 'Default'}


/home/dev/.virtualenvs/nfl-data-models/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/dev/.virtualenvs/nfl-data-models/lib/python3.10/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [2]:
experiment_description = (
    "NFL Win Probability Models"
    "This experiment contains models for neutral win probability as well as Vegas line adjusted win probability."
)

experiment_tags = {
    "project_name": "wp-models",
    "team": "nfl-ml",
    "mlflow.note.content": experiment_description,
}

nfl_wp_experiment = client.create_experiment(name="WP_Models", tags=experiment_tags)

wp_experiment = client.search_experiments(
    filter_string="tags.`project_name` = 'wp-models'"
)

pprint(wp_experiment[0])

<Experiment: artifact_location='mlflow-artifacts:/738046608083083887', creation_time=1700088617418, experiment_id='738046608083083887', last_update_time=1700088617418, lifecycle_stage='active', name='WP_Models', tags={'mlflow.note.content': 'NFL Win Probability ModelsThis experiment contains '
                        'models for neutral win probability as well as Vegas '
                        'line adjusted win probability.',
 'project_name': 'wp-models',
 'team': 'nfl-ml'}>


In [3]:
# Access individual tag data
print(wp_experiment[0].tags["team"])

nfl-ml


In [4]:
from src import WPModel

wp_model = WPModel()
cal_data = wp_model.import_calibration_data()
test_df = cal_data.loc[cal_data['season'] >= 2023]

In [5]:
import mlflow

# Use the fluent API to set the tracking uri and the active experiment
mlflow.set_tracking_uri("http://127.0.0.1:8080")

# Sets the current active experiment to the "Apple_Models" experiment and returns the Experiment metadata
apple_experiment = mlflow.set_experiment("WP_Models")

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "wp_model_xgb_custom_train_test_split"

# Define an artifact path that the model will be saved to.
artifact_path = "wp_model_xgb_custom_train_test_split"

# Model Results: XGBoost with custom train_test_split

In [7]:
import numpy as np

X_train, y_train, X_test, y_test, folds = wp_model.train_test_split(cal_data,2023, 5)
clf = wp_model.train_xgclassifier(X_train, y_train, X_test, y_test)
scores = clf.evals_result()
y_val = clf.predict(X_test)
y_pred = clf.predict_proba(X_test, validate_features=True)
test_df['wp'] = y_pred[:,1]
cols = ['game_id', 'game_seconds_remaining', 'score_differential', 'yardline_100', 'win_probability']
test_df = test_df.filter(items=cols)

[0]	validation_0-logloss:0.67713	validation_0-auc:0.82905	validation_0-error:0.27677	validation_1-logloss:0.67701	validation_1-auc:0.83616	validation_1-error:0.26513
[50]	validation_0-logloss:0.47830	validation_0-auc:0.86121	validation_0-error:0.22791	validation_1-logloss:0.48822	validation_1-auc:0.85296	validation_1-error:0.22974
[100]	validation_0-logloss:0.45688	validation_0-auc:0.86309	validation_0-error:0.22628	validation_1-logloss:0.47072	validation_1-auc:0.85420	validation_1-error:0.22835
[150]	validation_0-logloss:0.45222	validation_0-auc:0.86456	validation_0-error:0.22496	validation_1-logloss:0.46771	validation_1-auc:0.85493	validation_1-error:0.22791
[200]	validation_0-logloss:0.45046	validation_0-auc:0.86532	validation_0-error:0.22438	validation_1-logloss:0.46685	validation_1-auc:0.85523	validation_1-error:0.22708
[250]	validation_0-logloss:0.44944	validation_0-auc:0.86581	validation_0-error:0.22404	validation_1-logloss:0.46672	validation_1-auc:0.85522	validation_1-error:0.2

In [11]:
from sklearn.metrics import accuracy_score
# Calculate error metrics
accuracy = accuracy_score(y_val, y_pred[:,1] > .5)
error = sum(scores['validation_1']['error']) / len(scores['validation_1']['error'])
auc = sum(scores['validation_1']['auc']) / len(scores['validation_1']['auc'])
logloss = sum(scores['validation_1']['logloss']) / len(scores['validation_1']['logloss'])

# Assemble the metrics we're going to write into a collection
metrics = {"accuracy": accuracy, "error": error, "auc": auc, "logloss": logloss}

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(wp_model.wp_spread_model_sklearn_parameters)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(sk_model=clf, input_example=X_test, artifact_path=artifact_path)

/home/dev/.virtualenvs/nfl-data-models/lib/python3.10/site-packages/mlflow/models/signature.py:351: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  input_schema = _infer_schema(input_example)
/home/dev/.virtualenvs/nfl-data-models/lib/python3.10/site-packages/mlflow/models/signature.py:362: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Pyth

# Model Results: xgboost with scikit-learn train_test_split

In [12]:
from sklearn.model_selection import train_test_split

# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "wp_model_xgb_sklearn_train_test_split"

# Define an artifact path that the model will be saved to.
artifact_path = "wp_model_xgb_sklearn_train_test_split"

In [13]:

wp_model = WPModel()
cal_data = wp_model.import_calibration_data()
X = cal_data.loc[:, ~cal_data.columns.isin(['season', 'game_id', 'label', 'home_team', 'away_team'])]
y = cal_data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

clf = wp_model.train_xgclassifier(X_train, y_train, X_test, y_test)
scores = clf.evals_result()

y_val = clf.predict(X_test)
y_pred = clf.predict_proba(X_test, validate_features=True)

[0]	validation_0-logloss:0.67714	validation_0-auc:0.82942	validation_0-error:0.28308	validation_1-logloss:0.67720	validation_1-auc:0.82861	validation_1-error:0.28494
[50]	validation_0-logloss:0.47842	validation_0-auc:0.86113	validation_0-error:0.22786	validation_1-logloss:0.47867	validation_1-auc:0.86086	validation_1-error:0.22843
[100]	validation_0-logloss:0.45706	validation_0-auc:0.86302	validation_0-error:0.22618	validation_1-logloss:0.45733	validation_1-auc:0.86273	validation_1-error:0.22640
[150]	validation_0-logloss:0.45243	validation_0-auc:0.86447	validation_0-error:0.22480	validation_1-logloss:0.45272	validation_1-auc:0.86417	validation_1-error:0.22519
[200]	validation_0-logloss:0.45070	validation_0-auc:0.86521	validation_0-error:0.22418	validation_1-logloss:0.45115	validation_1-auc:0.86480	validation_1-error:0.22472
[250]	validation_0-logloss:0.44960	validation_0-auc:0.86576	validation_0-error:0.22393	validation_1-logloss:0.45022	validation_1-auc:0.86525	validation_1-error:0.2

In [14]:
# Calculate error metrics
accuracy = accuracy_score(y_val, y_pred[:,1] > .5)
error = sum(scores['validation_1']['error']) / len(scores['validation_1']['error'])
auc = sum(scores['validation_1']['auc']) / len(scores['validation_1']['auc'])
logloss = sum(scores['validation_1']['logloss']) / len(scores['validation_1']['logloss'])

# Assemble the metrics we're going to write into a collection
metrics = {"accuracy": accuracy, "error": error, "auc": auc, "logloss": logloss}

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(wp_model.wp_spread_model_sklearn_parameters)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(sk_model=clf, input_example=X_test, artifact_path=artifact_path)

/home/dev/.virtualenvs/nfl-data-models/lib/python3.10/site-packages/mlflow/models/signature.py:351: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  input_schema = _infer_schema(input_example)
/home/dev/.virtualenvs/nfl-data-models/lib/python3.10/site-packages/mlflow/models/signature.py:362: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Pyth

# Model Results: Stratified split by game id

In [16]:
# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "wp_model_xgb_stratified_split_game_id"

# Define an artifact path that the model will be saved to.
artifact_path = "wp_model_xgb_stratified_split_game_id"

In [17]:
wp_model = WPModel()
cal_data = wp_model.import_calibration_data()
X = cal_data.loc[:, ~cal_data.columns.isin(['season', 'game_id', 'label', 'home_team', 'away_team'])]
y = cal_data['label']
groups = cal_data['game_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=groups)

clf = wp_model.train_xgclassifier(X_train, y_train, X_test, y_test)
scores = clf.evals_result()
y_val = clf.predict(X_test)
y_pred = clf.predict_proba(X_test, validate_features=True)

[0]	validation_0-logloss:0.67714	validation_0-auc:0.82926	validation_0-error:0.26146	validation_1-logloss:0.67714	validation_1-auc:0.82858	validation_1-error:0.26222
[50]	validation_0-logloss:0.47806	validation_0-auc:0.86147	validation_0-error:0.22745	validation_1-logloss:0.47922	validation_1-auc:0.86012	validation_1-error:0.22922
[100]	validation_0-logloss:0.45667	validation_0-auc:0.86333	validation_0-error:0.22576	validation_1-logloss:0.45840	validation_1-auc:0.86183	validation_1-error:0.22756
[150]	validation_0-logloss:0.45200	validation_0-auc:0.86477	validation_0-error:0.22459	validation_1-logloss:0.45406	validation_1-auc:0.86316	validation_1-error:0.22665
[200]	validation_0-logloss:0.45026	validation_0-auc:0.86550	validation_0-error:0.22400	validation_1-logloss:0.45253	validation_1-auc:0.86382	validation_1-error:0.22589
[250]	validation_0-logloss:0.44919	validation_0-auc:0.86603	validation_0-error:0.22362	validation_1-logloss:0.45162	validation_1-auc:0.86428	validation_1-error:0.2

In [18]:
# Calculate error metrics
accuracy = accuracy_score(y_val, y_pred[:,1] > .5)
error = sum(scores['validation_1']['error']) / len(scores['validation_1']['error'])
auc = sum(scores['validation_1']['auc']) / len(scores['validation_1']['auc'])
logloss = sum(scores['validation_1']['logloss']) / len(scores['validation_1']['logloss'])

# Assemble the metrics we're going to write into a collection
metrics = {"accuracy": accuracy, "error": error, "auc": auc, "logloss": logloss}

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(wp_model.wp_spread_model_sklearn_parameters)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(sk_model=clf, input_example=X_test, artifact_path=artifact_path)

/home/dev/.virtualenvs/nfl-data-models/lib/python3.10/site-packages/mlflow/models/signature.py:351: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  input_schema = _infer_schema(input_example)
/home/dev/.virtualenvs/nfl-data-models/lib/python3.10/site-packages/mlflow/models/signature.py:362: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Pyth

# Model Results: GroupKFold

In [19]:
# Define a run name for this iteration of training.
# If this is not set, a unique name will be auto-generated for your run.
run_name = "wp_model_xgb_groupkfold"

# Define an artifact path that the model will be saved to.
artifact_path = "wp_model_xgb_groupkfold"

In [20]:
from sklearn.model_selection import GroupKFold

wp_model = WPModel()
cal_data = wp_model.import_calibration_data()
X = cal_data.loc[:, ~cal_data.columns.isin(['season', 'game_id', 'label', 'home_team', 'away_team'])]
y = cal_data['label']
groups = cal_data['game_id']

group_fold = GroupKFold(n_splits=5)
for train_index, test_index in group_fold.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

clf = wp_model.train_xgclassifier(X_train, y_train, X_test, y_test)
scores = clf.evals_result()
y_val = clf.predict(X_test)
y_pred = clf.predict_proba(X_test, validate_features=True)

[0]	validation_0-logloss:0.67739	validation_0-auc:0.82634	validation_0-error:0.26525	validation_1-logloss:0.67654	validation_1-auc:0.84121	validation_1-error:0.25253
[50]	validation_0-logloss:0.48214	validation_0-auc:0.85813	validation_0-error:0.23143	validation_1-logloss:0.46683	validation_1-auc:0.87317	validation_1-error:0.21397
[100]	validation_0-logloss:0.46103	validation_0-auc:0.86008	validation_0-error:0.22949	validation_1-logloss:0.44295	validation_1-auc:0.87443	validation_1-error:0.21341
[150]	validation_0-logloss:0.45647	validation_0-auc:0.86153	validation_0-error:0.22814	validation_1-logloss:0.43828	validation_1-auc:0.87532	validation_1-error:0.21197
[200]	validation_0-logloss:0.45465	validation_0-auc:0.86233	validation_0-error:0.22760	validation_1-logloss:0.43687	validation_1-auc:0.87570	validation_1-error:0.21157
[250]	validation_0-logloss:0.45370	validation_0-auc:0.86278	validation_0-error:0.22730	validation_1-logloss:0.43631	validation_1-auc:0.87586	validation_1-error:0.2

In [21]:
# Calculate error metrics
accuracy = accuracy_score(y_val, y_pred[:,1] > 0.5)
error = sum(scores['validation_1']['error']) / len(scores['validation_1']['error'])
auc = sum(scores['validation_1']['auc']) / len(scores['validation_1']['auc'])
logloss = sum(scores['validation_1']['logloss']) / len(scores['validation_1']['logloss'])

# Assemble the metrics we're going to write into a collection
metrics = {"accuracy": accuracy, "error": error, "auc": auc, "logloss": logloss}

# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(wp_model.wp_spread_model_sklearn_parameters)

    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)

    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(sk_model=clf, input_example=X_test, artifact_path=artifact_path)

/home/dev/.virtualenvs/nfl-data-models/lib/python3.10/site-packages/mlflow/models/signature.py:351: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  input_schema = _infer_schema(input_example)
/home/dev/.virtualenvs/nfl-data-models/lib/python3.10/site-packages/mlflow/models/signature.py:362: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Pyth